In [1]:
import pandas as pd
from pathlib import Path
import eyed3
import os
from config import *

notebook_path = os.path.abspath("./")

eyed3.log.setLevel("ERROR")

In [2]:
def make_file_list(root_dir):
    '''given a directory, loops through folders and files and return list of files paths
        and file names'''
    path_list = []
    for subdir, dirs, files in os.walk(root_dir):
        for file in files:
            if file.endswith(('.mp3', '.wav')):
                path_list.append(os.path.join(subdir, file))
            else:
                print('does not')
        #path_list = [os.path.join(subdir, file) for file in files if file.endswith(('.mp3', '.wav'))]
    return path_list

def get_genre_id(genre_dict_list):
    return genre_dict_list[0]['genre_id']

def id_to_int(val):
    return int(val)


In [3]:
metadata = notebook_path + "/fma_metadata"
os.chdir(metadata)
!pwd
genres_df = pd.read_csv('genres.csv')
tracks_df = pd.read_csv('raw_tracks.csv')

/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_metadata


In [4]:
# drop rows missing the 'genre' value
tracks_df = tracks_df[tracks_df['track_genres'].notna()]

# string conversion to list of genre dictionaries
tracks_df['track_genres'] = tracks_df['track_genres'].apply(eval)

# assign primary genre classification id, then make an int
tracks_df['genre_ids'] = tracks_df['track_genres'].apply(get_genre_id)

# convert IDs to ints for comparisons
tracks_df['genre_ids'] = tracks_df['genre_ids'].apply(id_to_int)
tracks_df['track_id'] = tracks_df['track_id'].apply(id_to_int)



In [5]:
# dictionaries for top-level genre lookup from ID values
genre_lookup = genres_df.set_index('genre_id').to_dict()['title']
id_lookup = genres_df.set_index('title').to_dict()['genre_id']

# if we ever want everything under only a few genres
# key = current genre, value = parent genre
parent_lookup = genres_df.set_index('genre_id').to_dict()['top_level']

# when a genre has no parent, it is top-level
top_genres = genres_df[genres_df['parent'] == 0]['title'].to_list()

In [7]:
# get songs to count genres
root_dir = notebook_path + "/fma_small"

song_paths = make_file_list(root_dir)

In [8]:
print(root_dir)
print(song_paths)

_small/114/114393.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114556.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114232.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114385.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114391.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114408.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114409.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114390.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114384.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114541.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114233.mp3', '/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/114/114974.mp3', '/Users/

In [10]:
# genre parent directory
gp_dir = notebook_path + "/genres"
print(gp_dir)

/Users/katie/code/capstone/Music_Genre_Classification/Katie/genres


In [11]:
try:
    Path(gp_dir).mkdir(mode=0o777, parents=False, exist_ok=True)
except ValueError:
    print('did not make genre dir')

In [12]:
# create directories
for genre in top_genres:
    if "/" in genre:
        genre = genre.replace('/', '&')
    genre_dir = gp_dir + "/" + genre
    # print(genre_dir)
    Path(genre_dir).mkdir(mode=0o777, parents=False, exist_ok=True)

In [14]:
moved = 0
# start counting genres
print(song_paths)
for i in range(len(song_paths)):
    # load audio file
    audiofile = eyed3.load(song_paths[i])
    filename = song_paths[i][-10:]

    # get genre and strip genre ID
    song_genre = str(audiofile.tag.genre)
    # will either be 0 or idx after (id)
    end = song_genre.find(')') + 1
    # for Romany(gypsy) genre
    if end == len(song_genre):
        end = 0
    song_genre = song_genre[end:]

    # change to top-level genre
    song_genre = genre_lookup[parent_lookup[id_lookup[song_genre]]]

    # fix for directory moving
    if "/" in song_genre:
        song_genre = song_genre.replace('/', '&')
    
    # rename and move files
    new_path = notebook_path + "/genres/" + song_genre + "/" + filename
    print(song_paths[i], new_path)
    os.replace(song_paths[i], new_path)
    moved += 1
print('moved ' + str(moved) + ' songs.')

ode/capstone/Music_Genre_Classification/Katie/fma_small/113/113789.mp3 /Users/katie/code/capstone/Music_Genre_Classification/Katie/genres/Instrumental/113789.mp3
/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/113/113969.mp3 /Users/katie/code/capstone/Music_Genre_Classification/Katie/genres/Instrumental/113969.mp3
/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/113/113028.mp3 /Users/katie/code/capstone/Music_Genre_Classification/Katie/genres/Pop/113028.mp3
/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/113/113564.mp3 /Users/katie/code/capstone/Music_Genre_Classification/Katie/genres/Experimental/113564.mp3
/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/113/113558.mp3 /Users/katie/code/capstone/Music_Genre_Classification/Katie/genres/Experimental/113558.mp3
/Users/katie/code/capstone/Music_Genre_Classification/Katie/fma_small/113/113809.mp3 /Users/katie/code/capstone/Music_Genre_Classification/Kati